In [1]:
pip install transformers datasets sentencepiece accelerate

  Using cached sentencepiece-0.2.0.tar.gz (2.6 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [48 lines of output]
      Traceback (most recent call last):
        File "c:\Users\user\poetic2\.venv\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 389, in <module>
          main()
          ~~~~^^
        File "c:\Users\user\poetic2\.venv\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 373, in main
          json_out["return_val"] = hook(**hook_input["kwargs"])
                                   ~~~~^^^^^^^^^^^^^^^^^^^^^^^^
        File "c:\Users\user\poetic2\.venv\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 143, in get_requires_for_build_wheel
          return hook(config_settings)
        File "C:\Users\user\AppData\Local\Temp\pip-build-env-1giq82p0\overlay\Lib\site-packages\setuptools\build_meta.py", line 331, in get_requires_for_build_wheel
         

In [3]:
# Load and parse Nepali poem dataset
def load_poems(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        content = f.read()

    # Split poems using two or more newlines
    poems = [p.strip() for p in content.strip().split('\n\n\n') if p.strip()]
    return poems

# Extract and print the first poem
def print_first_poem(file_path):
    poems = load_poems(file_path)
    if poems:
        print(poems[0])
    else:
        print("No poems found.")

# Example usage
print_first_poem('allpoems.txt')  # Replace with your dataset filename

Title:पाप लाग्छ
Author:लक्ष्मीप्रसाद देवकोटा
नटिप्नु हेर कोपिला!
नचुँड्नु पाप लाग्दछ।
नच्यात्नु फूल नानि हो!
दया र धर्म भाग्दछ।।

नछोप्नु है चरी बरी
सराप आँसु लाग्दछ।
नमार्नु जन्तु है कुनै
बसेर काल जाग्दछ।।

न घाउ चोट लाउनू
सडेर चित्त पाक्दछ।
धूलो नफेक्नु नानि हो!
उडेर भित्र ढाक्दछ।।

हिलो नखेल्नु फोहरी
खराब दाग लाग्दछ।
न चित्त है दुखाउनु
डसेर आँसु बग्दछ।।

बनेर फूल झैं सधैं
हँसाउनू सुवास दी।
सधैं रमाउनू जगत्
रमेर नित्य आश दी।।

जताततै छ ईश रे
छ सुन्नु त्यो विचार रे।
छकाउने लुकाउने
नराख भाव क्यै गरे!


In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM
#Loading the model and tokenizer
model_name = "Sakonii/distilgpt2-nepali"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

c:\Users\user\poetic2\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
!pip install datasets
from datasets import load_dataset

def tokenize_function(examples):
    return tokenizer(examples['text'], truncation=True, max_length=512)

# Load the dataset using the 'datasets' library
dataset = load_dataset('text', data_files='allpoems.txt')

# Now you can tokenize the dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True, remove_columns=["text"])

In [6]:
from transformers import TextDataset

def load_dataset(file_path, tokenizer, block_size=128):
    return TextDataset(
        tokenizer=tokenizer,
        file_path=file_path,
        block_size=block_size
    )

train_dataset = load_dataset("allpoems.txt", tokenizer)

c:\Users\user\poetic2\.venv\Lib\site-packages\transformers\data\datasets\language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [7]:
from transformers import DataCollatorForLanguageModeling


def load_data_collator(tokenizer, mlm = False):
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=mlm,
    )
    return data_collator

In [8]:
# you need to set parameters
train_file_path = "allpoems.txt"
model_name = 'Sakonii/distilgpt2-nepali'
output_dir="./distilgpt2-nepali-poem",
overwrite_output_dir = False
per_device_train_batch_size = 8
num_train_epochs = 250
save_steps = 500

In [9]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer # Importing TrainingArguments and Trainer

def train(train_file_path,model_name,
          output_dir,
          overwrite_output_dir,
          per_device_train_batch_size,
          num_train_epochs,
          save_steps):
  tokenizer = AutoTokenizer.from_pretrained(model_name)
  train_dataset = load_dataset(train_file_path, tokenizer)
  data_collator = load_data_collator(tokenizer)

  tokenizer.save_pretrained(output_dir)

  ### Loads model for finetuning
  model = AutoModelForCausalLM.from_pretrained(model_name)

  model.save_pretrained(output_dir)

  training_args = TrainingArguments( # TrainingArguments was not imported hence the error
          output_dir=output_dir,
          overwrite_output_dir=overwrite_output_dir,
          per_device_train_batch_size=per_device_train_batch_size,
          num_train_epochs=num_train_epochs,
      )

  trainer = Trainer( # Trainer was also not imported and needs to be imported
          model=model,
          args=training_args,
          data_collator=data_collator,
          train_dataset=train_dataset,
  )

  trainer.train()
  trainer.save_model()

In [10]:
pip install accelerate

Note: you may need to restart the kernel to use updated packages.


In [11]:
train(
    train_file_path = "allpoems.txt",
    model_name = 'Sakonii/distilgpt2-nepali',
    output_dir="./distilgpt2-nepali-poem",
    overwrite_output_dir = False,
    per_device_train_batch_size = 8,
    num_train_epochs = 250,
    save_steps = 500
)


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
500,4.826200
1000,3.227700
1500,2.009900
2000,1.144300
2500,0.640800
3000,0.378900
3500,0.249600
4000,0.193200
4500,0.163100
5000,0.141500


In [ ]:
import shutil
shutil.rmtree('distilgpt2-nepali-poem/checkpoint-1000')

In [12]:
def load_model(model_path):
    model = AutoModelForCausalLM.from_pretrained(model_path)
    return model


def load_tokenizer(tokenizer_path):
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
    return tokenizer


def generate_poem(sequence, max_length):
    model_path = "C:/Users/user/poetic2/distilgpt2-nepali-poem"
    model = load_model(model_path)
    tokenizer = load_tokenizer(model_path)
    ids = tokenizer.encode(f'{sequence}', return_tensors='pt')
    final_outputs = model.generate(
        ids,
        do_sample=True,
        max_length=max_length,
        pad_token_id=model.config.eos_token_id,
        top_k=50,
        top_p=0.95,
    )
    print(tokenizer.decode(final_outputs[0], skip_special_tokens=True))

In [23]:
sequence = input()
max_len = int(input())
generate_poem(sequence, max_len)

म र भरिया घडी साह राम्रा सुघर सुर जस्ता शहरिया । घडी घाँसी ग्वाला, कृषकहरु झुत्रै नगिचमा थियेँ साक्षी जस्तो सकल बटुवाको म बिचमा ॥ भिकी सुस्केराको बहुत खिरिलो सूत उसमा उनी माला मोतीसदृश पसिनाको दिवसमा । मलाई दिन्थे जो पथिकहरु ती बीच पथमा सबै शोची (धी) लिन्थेँ म पनि गहना त्यो सुपथमा ॥ जती प्यु
